<p style='color:red;'> Input Details </p>

- All end dates (Train, Test and Forecast)
- Specify date format
- Provide data file name (file should be in csv format)

## Import modules

In [1]:
import warnings, itertools, logging
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from tbats import TBATS

warnings.filterwarnings('ignore')
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

## Define dates

In [2]:
trainStartDate = datetime(2018,1,1)
trainEndDate = datetime(2022,1,1)
testStartDate = datetime(2022,2,1)
testEndDate = datetime(2023,1,1)
forecastStartDate = datetime(2023, 2, 1)
forecastEndDate = datetime(2024, 2, 1)

## Necessary functions for data manipulation

In [3]:
# Outlier detection
def detect_outlier(x):
    Q1 = np.quantile(x, 0.25)
    Q3 = np.quantile(x, 0.75)
    IQR = Q3 - Q1
    return (x < Q1 - (1.5*IQR)) | (x > Q3 + (1.5*IQR))
# Capping outlier value
def cap_outliers(x):
    Q1 = np.quantile(x, 0.25)
    Q3 = np.quantile(x, 0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - (1.5*IQR)
    upper_limit = Q3 + (1.5*IQR)
    x_cap = np.where(x > upper_limit, upper_limit, np.where(x < lower_limit, lower_limit, x))
    return x_cap
# MAPE (Mean Absolute Percentage Error)
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# Accuracy
def Accuracy(mape):
    return 100.0 - mape
# Root Mean Square Error
def RMSE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred)**2))

# Making function for data processing
# ----------------------------------------------------------------------------
def data_part(df, part):
    Date = pd.date_range(trainStartDate - timedelta(days=1), testEndDate, freq = 'M') + timedelta(days = 1)
    P1 = df[df['Part'] == part].reset_index(drop=True)
    P2 = P1.groupby(['Date'])['Quantity'].sum().reset_index()
    P2.set_index('Date', inplace = True)
    temp_df = pd.DataFrame(P2, index = Date)
    temp_df['Flag NA'] = temp_df['Quantity'].apply(lambda x: np.isnan(x))
    temp_df = temp_df[temp_df[temp_df['Flag NA'] == False].index[0]:]
    temp_df.fillna(method = 'ffill', inplace = True)
    temp_df['Outlier'] = detect_outlier(temp_df['Quantity'])
    temp_df['Quantity'] = cap_outliers(temp_df['Quantity'].values)
    return temp_df
# ADF test
def adf_test(series):
    result = adfuller(series)
    if result[1] < 0.05:
        d = 0
    else:
        d = 1
    return d

## Functions for different models

In [4]:
# ===========================================================================================================
#                                        Halt Winter
# -----------------------------------------------------------------------------------------------------------
def RUN_HW(df, part):
    best_rmse = float("inf")
    best_comb = None
    best_abg = None
    best_mape = None
    best_acc = None
    best_pred = None

    # Getting data for each group
    demo_data = data_part(df, part)
    # Data splitting
    train_data = demo_data[:trainEndDate]
    test_data = demo_data[trainEndDate + timedelta(days = 1):testEndDate]
    forecast_dates = pd.date_range(testEndDate, forecastEndDate, freq = 'M') + timedelta(days = 1)
    n = len(test_data)
    m = len(forecast_dates)
    y_true = test_data['Quantity']
    # Making all possible combination
    values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    all_combination = list(itertools.product(['add', 'mul'], ['add', 'mul']))
    abg_combination = list(itertools.product(values, values, values))

    for comb in tqdm(all_combination, desc = f'\nPart {part}', leave = False):
        for abg in tqdm(abg_combination, desc = 'Searching for best ABG ', leave = False):
            alpha, beta, gamma = abg
            try:
                hw_model = ExponentialSmoothing(train_data["Quantity"], trend=comb[0], seasonal=comb[1], seasonal_periods=12,damped = False)
                model = hw_model.fit(smoothing_level=alpha, smoothing_slope=beta, smoothing_seasonal=gamma)
                y_pred = model.predict(start=test_data.index[0], end=test_data.index[-1])
                mape = MAPE(y_true, y_pred)
                acc = Accuracy(mape)
                rmse = RMSE(y_true, y_pred)
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_comb = comb
                    best_abg = abg
                    best_mape = mape
                    best_acc = acc
                    best_pred = y_pred
            except:
                pass
    hw_final_model = ExponentialSmoothing(train_data["Quantity"], trend=best_comb[0], seasonal=best_comb[1], seasonal_periods=12,damped = False)
    final_model = hw_final_model.fit(smoothing_level = best_abg[0], smoothing_slope = best_abg[1], smoothing_seasonal = best_abg[2])
    forecast = final_model.predict(start = testEndDate+timedelta(days=1), end = forecastEndDate)
    temp = pd.DataFrame({'Part':np.repeat(part,n), 
                         'Date':test_data.index.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':best_pred.values, 
                         'MAPE':np.repeat(best_mape,n), 
                         'RMSE':np.repeat(best_rmse,n), 
                         'Accuracy':np.repeat(best_acc,n)})
    temp.insert(6, 'Order', [(best_comb, best_abg) for i in range(n)])
    forecast_df = pd.DataFrame({'Part':np.repeat(part,m), 
                                'Date':forecast_dates.values, 
                                'Forecast':forecast.values})
    forecast_df.insert(3, 'Order', [(best_comb, best_abg) for i in range(m)])
    return temp, forecast_df

# ===========================================================================================================
#                          Auto Regressive Moving Average (ARMA)
# -----------------------------------------------------------------------------------------------------------
def RUN_ARMA(df, part):
    best_rmse = float("inf")
    best_order = None
    best_mape = None
    best_acc = None
    best_pred = None

    # Getting data for each group
    demo_data = data_part(df, part)

    # Data splitting
    train_data = demo_data[:trainEndDate]
    test_data = demo_data[trainEndDate + timedelta(days = 1):testEndDate]
    forecast_dates = pd.date_range(testEndDate, forecastEndDate, freq = 'M') + timedelta(days = 1)
    n = len(test_data)
    m = len(forecast_dates)
    y_true = test_data['Quantity']
    
    p = range(0, 12)
    d = [0]
    q = range(0, 12)
    pdq_combination = list(itertools.product(p,d,q))
    
    for order in tqdm(pdq_combination, desc = f'\nPart: {part}', leave = False):
        try:
            model = ARIMA(train_data['Quantity'], order=order).fit()
            y_pred = model.predict(start=test_data.index[0], end=test_data.index[-1])
            mape = MAPE(y_true, y_pred)
            acc = Accuracy(mape)
            rmse = RMSE(y_true, y_pred)
            if rmse < best_rmse:
                best_rmse = rmse
                best_order = order
                best_mape = mape
                best_acc = acc
                best_pred = y_pred
        except:
            pass
    final_model = ARIMA(train_data['Quantity'], order = best_order).fit()
    forecast = final_model.predict(start = testEndDate + timedelta(days=1), end = forecastEndDate)
    temp = pd.DataFrame({'Part':np.repeat(part,n),
                         'Date':test_data.index.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':best_pred.values, 
                         'MAPE':np.repeat(best_mape,n), 
                         'RMSE':np.repeat(best_rmse,n), 
                         'Accuracy':np.repeat(best_acc,n)})
    temp.insert(6, 'Order', [best_order for i in range(n)])
    
    forecast_df = pd.DataFrame({'Part':np.repeat(part,m),
                         'Date':forecast_dates.values, 
                         'Forecast':forecast.values})
                                
    forecast_df.insert(3, 'Order', [best_order for i in range(m)])
    return temp, forecast_df


# ===========================================================================================================
#                                  Auto Regressive Integrated Moving Average (ARIMA)
# -----------------------------------------------------------------------------------------------------------
def RUN_ARIMA(df, part):
    best_rmse = float("inf")
    best_order = None
    best_mape = None
    best_acc = None
    best_pred = None
    # Getting data for each group
    demo_data = data_part(df, part)
    
    # Data splitting
    train_data = demo_data[:trainEndDate]
    test_data = demo_data[trainEndDate + timedelta(days = 1):testEndDate]
    forecast_dates = pd.date_range(testEndDate, forecastEndDate, freq = 'M') + timedelta(days = 1)
    n = len(test_data)
    m = len(forecast_dates)
    y_true = test_data['Quantity']
    
    # Defining possible combination set
    p = range(0, 10)
    d = [adf_test(demo_data['Quantity'])]
    q = range(0, 10)
    pdq_combination = list(itertools.product(p,d,q))
    
    for pdq in tqdm(pdq_combination, desc = f'\nPart: {part}', leave = False):
        try:
            model = ARIMA(train_data['Quantity'], order=pdq).fit()
            y_pred = model.predict(start=test_data.index[0], end=test_data.index[-1])
            mape = MAPE(y_true, y_pred)
            acc = Accuracy(mape)
            rmse = RMSE(y_true, y_pred)
            if rmse < best_rmse:
                best_rmse = rmse
                best_order = pdq
                best_mape = mape
                best_acc = acc
                best_pred = y_pred
        except:
            pass
        
    # final model
    ARIMA_final_model = ARIMA(train_data['Quantity'], order=best_order).fit()
    forecast = ARIMA_final_model.predict(start = testEndDate + timedelta(days=1), end = forecastEndDate)
    temp = pd.DataFrame({'Part':np.repeat(part,n), 
                         'Date':test_data.index.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':best_pred.values, 
                         'MAPE':np.repeat(best_mape,n), 
                         'RMSE':np.repeat(best_rmse,n), 
                         'Accuracy':np.repeat(best_acc,n)})
    temp.insert(6, 'Order', [best_order for i in range(n)])


    forecast_df = pd.DataFrame({'Part':np.repeat(part,m), 
                         'Date':forecast_dates.values, 
                         'Forecast':forecast.values})
    forecast_df.insert(3, 'Order', [best_order for i in range(m)])
    return temp, forecast_df

# ===========================================================================================================
#                          Seasonal Auto Regressive Integrated Moving Average (SARIMA)
# -----------------------------------------------------------------------------------------------------------
def RUN_SARIMA(df, part):
    best_rmse = float("inf")
    best_pdq = None
    best_pdqs = None
    best_mape = None
    best_acc = None
    best_pred = None

    # Getting data for each group
    demo_data = data_part(df, part)

    # Data splitting
    train_data = demo_data[:trainEndDate]
    test_data = demo_data[trainEndDate + timedelta(days = 1):testEndDate]
    forecast_dates = pd.date_range(testEndDate, forecastEndDate, freq = 'M') + timedelta(days = 1)
    n = len(test_data)
    m = len(forecast_dates)
    y_true = test_data['Quantity']

    #All possible combinations
    p = range(0,3)
    d = [adf_test(demo_data['Quantity'])]
    q = range(0, 3)
    P = range(0, 3)
    D = [0,1]
    Q = range(0, 3)
    s = [3, 6, 12]
    pdq_combination = list(itertools.product(p,d,q))
    PDQs_combination = list(itertools.product(P, D, Q, s))
    
    for pdq in tqdm(pdq_combination, desc = f'\nPart {part}', leave = False):
        for PDQs in tqdm(PDQs_combination, desc = 'Searching for best combination', leave = False):
            try:
                model = SARIMAX(train_data['Quantity'], order = pdq, seasonal_order = PDQs).fit()
                y_pred = model.predict(start = test_data.index[0], end = test_data.index[-1])
                y_true = test_data['Quantity']
                mape = MAPE(y_true, y_pred)
                acc = Accuracy(mape)
                rmse = RMSE(y_true, y_pred)
                
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_mape = mape
                    best_acc = acc
                    best_pred = y_pred
                    best_pdq = pdq
                    best_pdqs = PDQs
            except:
                pass
    
    temp = pd.DataFrame({'Part':np.repeat(part,n), 
                         'Date':test_data.index.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':best_pred.values, 
                         'MAPE':np.repeat(best_mape,n), 
                         'RMSE':np.repeat(best_rmse,n), 
                         'Accuracy':np.repeat(best_acc,n)})
    temp.insert(6, 'Order', [(best_pdq, best_pdqs) for i in range(n)])
    
    # Forecasting
    final_model = SARIMAX(train_data['Quantity'], order = best_pdq, seasonal_order = best_pdqs).fit()
    s = testEndDate + timedelta(days=1)
    e = forecastEndDate
    forecast = final_model.predict(start=s, end=e)
    forecast_df = pd.DataFrame({'Part':np.repeat(part,m),
                                "Date": forecast_dates.values ,
                                "Forecast": forecast.values})
    forecast_df.insert(3, 'Order', [(best_pdq, best_pdqs) for i in range(m)])
    
    return temp, forecast_df

# ===========================================================================================================
#                                                   TBATS
# -----------------------------------------------------------------------------------------------------------
def RUN_TBATS(df, part):
    best_rmse = float("inf")
    best_sp = None
    best_mape = None
    best_acc = None
    best_pred = None

    # Getting data for each group
    demo_data = data_part(df, part)
    sp_values = [3, 4, 5, 6, 8, 9, 12]

    # Data splitting
    train_data = demo_data[:trainEndDate]
    test_data = demo_data[trainEndDate + timedelta(days = 1):testEndDate]
    forecast_dates = pd.date_range(forecastStartDate - timedelta(days = 1), forecastEndDate, freq='M') + timedelta(days = 1)
    n = len(test_data)
    m = len(forecast_dates)
    y_true = test_data['Quantity']
    
    for sp in tqdm(sp_values, desc = 'Searching over possible seasonal periods', leave = False):
        try:
            model = TBATS(seasonal_periods=[sp]).fit(train_data.Quantity)
            y_pred = model.forecast(steps=n)
            mape = MAPE(y_true, y_pred)
            acc = Accuracy(mape)
            rmse = RMSE(y_true, y_pred)
            if rmse < best_rmse:
                best_rmse = rmse
                best_sp = sp
                best_mape = mape
                best_acc = acc
                best_pred = y_pred
        except:
            pass
    
    final_model = TBATS(seasonal_periods=[best_sp]).fit(train_data.Quantity) 
    y_forecast = final_model.forecast(steps=12+m)
    
    temp_df = pd.DataFrame({'Part':np.repeat(part,n), 
                         'Date':test_data.index.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':[best_pred[i] for i in range(0, n)], 
                         'MAPE':np.repeat(best_mape,n), 
                         'RMSE':np.repeat(best_rmse,n), 
                         'Accuracy':np.repeat(best_acc,n)})
    temp_df.insert(6, 'Order', [(best_sp) for i in range(n)])
    
    forecast_df = pd.DataFrame({'Part':np.repeat(part, m), 
                         'Date':forecast_dates.values, 
                         'Forecast':[y_forecast[n-1+i] for i in range(0, m)]})
    forecast_df.insert(3, 'Order', [(best_sp) for i in range(m)])
    
    return temp_df, forecast_df

# ----------------------------------------------------------------------------
#                                   Prophet
# ----------------------------------------------------------------------------
def RUN_PROPHET(df, part):
    rmse = float("inf")
    mape = None
    acc = None

    # Getting data for each group
    demo_data = data_part(df, part)
    demo_data.reset_index(level=0, inplace=True)
    demo_data.columns = ['Date', 'Quantity', 'Flag NA', 'Outlier']

    # Data splitting
    testEndDate = demo_data.index.values[-1]
    train_data = demo_data.iloc[:demo_data.index[demo_data.Date == trainEndDate].values[0]]
    test_data = demo_data.iloc[demo_data.index[demo_data.Date == trainEndDate].values[0]+1:testEndDate]
    train_data = train_data[["Date", "Quantity"]]
    train_data.columns = ['ds', 'y']
    test_period = pd.DataFrame(test_data["Date"])
    test_period.columns = ['ds']
    y_true = test_data['Quantity']
    n = len(test_data)
    forecast_dates = pd.date_range(forecastStartDate - timedelta(days = 1), forecastEndDate , freq='M') + timedelta(days = 1)
    
    try:
        model = Prophet()
        model.fit(train_data)
        prediction = model.predict(test_period)
        y_pred = prediction["yhat"]
        mape = MAPE(y_true, y_pred)
        acc = Accuracy(mape)
        rmse = RMSE(y_true, y_pred)

    except:
        pass
    
    forecast_period = pd.DataFrame(forecast_dates)
    forecast_period.columns = ['ds']
    m = len(forecast_period)
    forecast = model.predict(forecast_period)
    y_forecast = forecast["yhat"]
    
    temp_df = pd.DataFrame({'Part':np.repeat(part,n), 
                         'Date':test_data.Date.values, 
                         'Original':test_data.Quantity.values, 
                         'Prediction':[y_pred[i] for i in range(0, n)], 
                         'MAPE':np.repeat(mape,n), 
                         'RMSE':np.repeat(rmse,n), 
                         'Accuracy':np.repeat(acc,n)})
    temp_df.insert(6, 'Order', [np.nan for i in range(n)])
    
    forecast_df = pd.DataFrame({'Part':np.repeat(part,m), 
                         'Date':forecast_period.ds.values, 
                         'Forecast':[y_forecast[i] for i in range(0, m)]})
    forecast_df.insert(3, 'Order', [np.nan for i in range(m)])
    
    return temp_df, forecast_df


## Run Model Function

In [5]:
def RUN_MODEL(date_format, data_file_name, model, model_fun):
    # Load the data and data processing ------------------------------------------
    data = pd.read_csv(f'./../data/{data_file_name}.csv', low_memory = False, encoding= 'unicode_escape')
    df = data[['Part','Customer Desc','Date','Quantity']]
    df["Date"] = pd.to_datetime(df["Date"], format = date_format)
    # Define SKU list ------------------------------------------------------------
    sku_set = df['Part'].unique()[:2]
    # Run all SKU for a specific model -------------------------------------------
    print(f'MODEL: {model} || Running for {len(sku_set)} parts: ')
    prediction_df = pd.DataFrame()
    forecast_df = pd.DataFrame()
    for part in tqdm(sku_set, desc = 'Over All Parts'):
        output = model_fun(df, part)
        prediction_df = pd.concat([prediction_df, output[0]], ignore_index = True)
        forecast_df = pd.concat([forecast_df, output[1]], ignore_index = True)

    summary_df = prediction_df[['Part','MAPE','RMSE','Accuracy','Order']]
    summary_df.drop_duplicates(inplace = True)
    summary_df.reset_index(drop = True, inplace = True)
    summary_df['Model'] = np.repeat('Model', len(sku_set))

    prediction_df.to_csv(f'./../result/prediction_{model}_{len(sku_set)}.csv', index = False)
    forecast_df.to_csv(f'./../result/forecast_{model}_{len(sku_set)}.csv', index = False)
    summary_df.to_csv(f'./../result/summary_{model}_{len(sku_set)}.csv', index = False)

    print(f"Average MAPE for {model}: {summary_df['MAPE'].mean()}\n")

## Finally Run All model

In [6]:
# Function input
# -----------------------------------------------------------------------------
date_format = '%d-%b-%y'
data_file_name = ''

RUN_MODEL(date_format, data_file_name, 'HW', RUN_HW)
RUN_MODEL(date_format, data_file_name, 'ARMA', RUN_ARMA)
RUN_MODEL(date_format, data_file_name, 'ARIMA', RUN_ARIMA)
RUN_MODEL(date_format, data_file_name, 'SARIMA', RUN_SARIMA)
RUN_MODEL(date_format, data_file_name, 'TBATS', RUN_TBATS)
RUN_MODEL(date_format, data_file_name, 'PROBHET', RUN_PROPHET)

MODEL: PROBHET || Running for 2 parts: 



Average MAPE for PROBHET: 20.695249800231757



## Merging the model output

In [ ]:
# # Define lists
# models = ['HW','ARMA','ARIMA','SARIMA','PROPHET','TBATS']

# # Define function to concate model wise table
# def concat_df(paths, model_name):
#     output = pd.DataFrame()
#     for path in paths:
#         temp = pd.concat([temp, pd.read_csv(path)])
#     output.reset_index(drop = True, inplace = True)
#     temp['Model'] = model_name
#     return temp

# # Collecting files paths
# path_files = []
# for model in models:
#     for part in parts:
#         path_files.append('./../result/results_' + model + str(part) + '.csv')

# # Read all those files pair-wise
# p = len(parts)
# hw = concat_df(path_files[0:p], 'HW')
# arma = concat_df(path_files[p:2*p], 'ARMA')
# arima = concat_df(path_files[2*p:3*p], 'ARIMA')
# sarima = concat_df(path_files[3*p:4*p], 'SARIMA')
# prophet = concat_df(path_files[4*p:5*p], 'PROPHET')
# tbats = concat_df(path_files[5*p:6*p], 'TBATS')

# # Let's concate everything
# results_all_model = pd.concat([hw, arma, arima, sarima, prophet, tbats])
# results_all_model.reset_index(drop = True, inplace = True)

# # Assigning volatility and priority
# volatile = []
# priority = []
# for part in results_all_model['Part']:
#     volatile.append(assign_volatility(part))
#     priority.append(assign_priority(part))
# results_all_model['Volatility'] = volatile
# results_all_model['Priority'] = priority

# # Store the file
# print(f'Shape: {results_all_model.shape} and shape should be ({sum(parts)*12*6}, {11})')
# results_all_model.to_csv('./../result/final_merged_prediction_305.csv', index = False)